# LGA Mapping - Australia

## Created by: Rebecca Jan
## Last updated: 2021-03-27

Set up notebook

In [69]:
import geopandas as gpd
from geopandas.tools import sjoin
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import Point,  Polygon

Set path and load files

In [70]:
path ='C:\\Users\\vicsxk6\\OneDrive - Department of Health and Human Services. Victoria\\Documents\\Python Scripts\\lga_mapping\\'
sitrep = pd.read_csv(path+"data\sitrep.csv", low_memory=False, sep =",")

AUS lookup sourced from - https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.003June%202020

In [71]:
aus_lga = gpd.read_file(path+"data\\reference\\aus_lga_polygon_shp\LGA_2020_AUST.shp")

Assign points to .csv data

In [72]:
geometry = [Point(xy) for xy in zip(sitrep["Longitude_yest"], sitrep["Latitude_yest"])]
geo_df = gpd.GeoDataFrame(sitrep, crs="EPSG:4326", geometry=geometry)

Run inner join to pull in .shp data based on point

In [73]:
aus_lga = aus_lga.to_crs(crs="EPSG:4326")
pointInPolys = sjoin(geo_df, aus_lga, how="left", op="within")

Sanity check data

In [75]:
pointInPolys.info()
#pointInPolys.LGA_NAME20.unique()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 68641 entries, 0 to 68640
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   AccountID_today         68641 non-null  object  
 1   AccountID_yest          68641 non-null  object  
 2   AddressLine1_today      68397 non-null  object  
 3   AddressLine1_yest       68315 non-null  object  
 4   AddressLine2_today      68641 non-null  object  
 5   AddressLine2_yest       68641 non-null  object  
 6   Classification_today    3323 non-null   object  
 7   Classification_yest     3190 non-null   object  
 8   FullCleanAddress_today  66011 non-null  object  
 9   FullCleanAddress_yest   65858 non-null  object  
 10  LGA_today               68641 non-null  object  
 11  LGA_yest                68641 non-null  object  
 12  Latitude_today          66032 non-null  float64 
 13  Latitude_yest           66032 non-null  float64 
 14  Longitude_toda

Drop columns as needed

In [77]:
pointInPolys.drop(pointInPolys.columns[[22,23,24,26,28]], axis = 1, inplace = True)

Convert columns to upper and export to csv

In [61]:
pointInPolys.columns= pointInPolys.columns.str.upper()
pointInPolys.to_csv(path+"output\lga_mapped.csv", index=False)